
# Load Data, Drop Unecessary Information

In [1]:
import pandas as pd
import os

In [2]:
def create_complete_df(num_files):
    path_to_features  = '/content/drive/MyDrive/SOC/Data/'

    list_of_dfs = []
    for i in range(num_files):
        csv_file = f'TOAFastFeatures{i + 1}.csv'
        print(csv_file)
        df = pd.read_csv(os.path.join(path_to_features, csv_file))
        list_of_dfs.append(df)

    df = pd.concat(list_of_dfs, axis=0)
    return df

In [3]:
df = create_complete_df(1)

TOAFastFeatures1.csv


In [5]:
df.reset_index(inplace = True)

In [15]:
df.drop(['index'], axis = 1, inplace = True)

# Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(verbose=2, max_iter=20)

imputed_df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
imputed_df

# Merge HWSD Data

In [ ]:
HWSD = pd.read_csv('/content/drive/MyDrive/SOCData/HWSD_DATA.csv')
HWSD = HWSD[['MU_GLOBAL', 'S_OC']]
HWSD

In [19]:
new = pd.merge(imputed_df, HWSD, on="MU_GLOBAL")
new = new.dropna()

# Split Data

In [21]:
y = new.pop('S_OC')
X = new

In [22]:
assert 'S_OC' not in X.columns

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [24]:
assert len(X_train) == len(y_train)

In [25]:
assert len(X_test) == len(y_test)

# Feature Standardization

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

# Random Forest Regressor

In [27]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

# RMSE

In [29]:
from sklearn.metrics import mean_squared_error

round(mean_squared_error(rf.predict(X_test), y_test), 3)

6.494